## Helper notebook for cropping photometry signals


Sometimes when recording photometry signals with LabVIEW, the signal cuts out and drops to zero! So we want to crop the signal to only valid recording times. Here we plot the raw signals to see where the drop happened, so we can set an appropriate `phot_end_time_mins` value in our metadata file.

To load your raw photometry signals, either specify your `metadata_file_path` (assuming your metadata file includes `phot_file_path` and `box_file_path`), or just set `phot_file_path` and `box_file_path` directly.

In [ ]:
import yaml
import numpy as np
import pandas as pd
from jdb_to_nwb.convert_photometry import process_raw_labview_photometry_signals
from jdb_to_nwb.plotting.plot_photometry import plot_photometry_signals
from jdb_to_nwb.utils import setup_stdout_logger

# Read metadata file to get phot and box file paths, or set phot_file_path and box_file_path directly
metadata_file_path = "/Users/steph/berkelab/jdb_to_nwb/metadata/metadata_IM-1875_20250718.yaml"
with open(metadata_file_path, "r") as f:
    metadata = yaml.safe_load(f)

phot_file_path = metadata["photometry"]["phot_file_path"] # read from metadata or set this directly
box_file_path = metadata["photometry"]["box_file_path"] # read from metadata or set this directly

# Setup dummy logger that just prints to stdout instead of a logfile
logger = setup_stdout_logger(log_name="phot_test_log")

# Read raw phot and box files into signals dict
signals = process_raw_labview_photometry_signals(phot_file_path, box_file_path, logger)

# Downsample the raw data from 10 kHz to 250 Hz by taking every 40th sample
SR = 10000  # Original sampling rate of the photometry system (Hz)
Fs = 250  # Target downsample frequency (Hz)
raw_reference = pd.Series(signals["ref"][:: int(SR / Fs)])
raw_green = pd.Series(signals["sig1"][:: int(SR / Fs)])
port_visits = np.divide(signals["visits"], SR / Fs).astype(int)


10-Sep-25 14:59:04 [INFO] Reading LabVIEW .phot file into a dictionary...
10-Sep-25 14:59:05 [DEBUG] Data read from LabVIEW .phot file at /Volumes/Tim/Photometry/IM-1875/20250718/IM-1875_2025-07-18_16-07-55____Tim_Conditioning.phot:
10-Sep-25 14:59:05 [DEBUG] magic_key: 22289481
10-Sep-25 14:59:05 [DEBUG] header_size: 20480
10-Sep-25 14:59:05 [DEBUG] main_version: 0
10-Sep-25 14:59:05 [DEBUG] secondary_version: 0
10-Sep-25 14:59:05 [DEBUG] sampling_rate: 10000
10-Sep-25 14:59:05 [DEBUG] bytes_per_sample: 2
10-Sep-25 14:59:05 [DEBUG] num_channels: 8
10-Sep-25 14:59:05 [DEBUG] file_name: C:\Users\BerkeLab\Documents\Labview\IM-1875\2025-07-18\IM-1875_2025-07-18_16-07-55____Tim_Conditioning.phot                                                                                                                                                     
10-Sep-25 14:59:05 [DEBUG] date: 2025-07-18                                                                                                           

Plot the raw photometry signals in interactive mode so we can zoom in and see where to crop

In [ ]:
# interactive mode!
%matplotlib tk

# Plot the raw signals so we can zoom in and see where to crop
fig = plot_photometry_signals(visits=port_visits,
                        sampling_rate=Fs,
                        signals=[raw_green, raw_reference],
                        signal_labels=["Raw 470nm signal", "Raw 405nm signal"],
                        signal_colors=["blue", "purple"],
                        title="Raw LabVIEW photometry signals",
                        signal_units="a.u.",
                        fig_dir=None)
fig.show()

Set `phot_end_time_mins` based on the above plot

In [ ]:
phot_end_time_mins = 77.6

Sanity check - plot the cropped photometry signals to check that we picked a good end time

In [ ]:
# Convert end time into number of photometry samples to keep
samples_to_keep = int(phot_end_time_mins * 60 * Fs)

# Plot the cropped signals to make sure the crop is good
raw_reference_cropped = raw_reference[:samples_to_keep]
raw_green_cropped = raw_green[:samples_to_keep]
fig = plot_photometry_signals(visits=port_visits,
                        sampling_rate=Fs,
                        signals=[raw_green_cropped, raw_reference_cropped],
                        signal_labels=["Raw 470nm signal", "Raw 405nm signal"],
                        signal_colors=["blue", "purple"],
                        title="Cropped photometry signals",
                        signal_units="a.u.",
                        fig_dir=None)
fig.show()

Now go set `phot_end_time_mins` in the your metadata file under `photometry` so the nwb conversion pipeline crops your signal to only the valid times :)